In [62]:
import pandas as pd
import numpy as np
import json 
import copy
import csv
import os
import re
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from nltk.corpus import stopwords
import sklearn


In [63]:
data = [] 
cf = open('bars_rev_tr_df.csv','r')

file = csv.DictReader(cf)
#print(file.fieldnames)

for x in file:
    line = {'business_id':x['business_id'],'date':x['date'],'stars':x['stars'],'text':x['text']}
    data.append(line)

cf.close()

id = open('uniq_id_lvbars.txt','r')
id_str=id.read()
id.close()

id_list=id_str.split(',')

dat_night=[]
for i in range(len(data)):
    if data[i]['business_id'] in id_list:
        dat_night.append(data[i])



dat_night_5star=[]
dat_night_4star=[]
dat_night_3star=[]
dat_night_2star=[]
dat_night_1star=[]
for i in range(len(dat_night)):
    if dat_night[i]['stars']=='5':
        dat_night_5star.append(dat_night[i])
    elif dat_night[i]['stars']=='4':
        dat_night_4star.append(dat_night[i])
    elif dat_night[i]['stars']=='3':
        dat_night_3star.append(dat_night[i])
    elif dat_night[i]['stars']=='2':
        dat_night_2star.append(dat_night[i])
    else:
        dat_night_1star.append(dat_night[i])


In [64]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)|(\%)|(\_)|(\=)|(\#)|(\&)|(\~)|(\@)")#[^\P{P}-]+
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    return reviews

In [65]:
def generate_ngrams(review, n):
    """
    Generate n-grams from a review
    :param review: one single review
    :param n: the size of n gram
    :return: generated list of n-grams
    """
    sentences = review.split(".")
    n_grams = []
    for sentence in sentences:
        sections = sentence.split(",")
        for section in sections:
            parts = section.split(";")
            for part in parts:
                words = part.split()
                
                n_grams_temp = []    # the return list
                for i in range(len(words)):
                    temp = words[i]
                    for j in range(1, n):
                        if (i + j) < len(words):
                            temp = temp + ' ' + words[i + j]
                            n_grams.append(temp)
    return n_grams

In [66]:
def bigramFreq(dat_night_star,xxx,n):
    reviewslist = copy.deepcopy(dat_night_star)


    reviewsTEXT_clean = copy.deepcopy(reviewslist)

    text_cloud1=''

    for ind in range(len(reviewslist)):
        texts = ''
        texts = reviewslist[ind]['text']
        texts = texts.lower()
    
        texts = re.sub('n\'t',' not', texts)
    
        texts = re.sub('isnt','isn\'t', texts)
        texts = re.sub('wasnt','wasn\'t', texts)
        texts = re.sub('werent','weren\'t', texts)
        texts = re.sub('dont','don\'t', texts)
        texts = re.sub('doesnt','doesn\'t', texts)
        texts = re.sub('didnt','didn\'t', texts)
        texts = re.sub('hasnt','hasn\'t', texts)
        texts = re.sub('havent','haven\'t', texts)
        texts = re.sub('hadnt','hadn\'t', texts)
        texts = re.sub('mightnt','mightn\'t', texts)
        texts = re.sub('shouldnt','shouldn\'t', texts)
        texts = re.sub('isn','isn\'t', texts)
        texts = re.sub('wasn','wasn\'t', texts)
        texts = re.sub('weren','weren\'t', texts)
        texts = re.sub('don','don\'t', texts)
        texts = re.sub('doesn','doesn\'t', texts)
        texts = re.sub('didn','didn\'t', texts)
        texts = re.sub('hasn','hasn\'t', texts)
        texts = re.sub('haven','haven\'t', texts)
        texts = re.sub('hadn','hadn\'t', texts)
        texts = re.sub('mightn','mightn\'t', texts)
        texts = re.sub('shouldn','shouldn\'t', texts)
        texts = re.sub('won\'t','will not', texts)
        texts = re.sub('n\'t',' not', texts)
    
        #add NOT_
        pattern = r'\.|\;|\!|\?|\,|\)|\(|\:|\'|\"|\%'
        list_text=re.split(pattern,texts)
    
        sent=''
        for i in range(len(list_text)):
            list_text[i] = re.sub('\+','', list_text[i])
            list_text[i] = re.sub('\*','', list_text[i])
            list_text[i] = re.sub('\$','', list_text[i])
            list_text[i] = re.sub('\[','', list_text[i])
            list_text[i] = re.sub('\]','', list_text[i])
            list_text[i] = re.sub('\%','', list_text[i])
            list_text[i] = re.sub('\\\\',' ', list_text[i])
            matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
            matchObj2 = re.search(r'(.*) never (.*)',list_text[i])
        
            if matchObj1 != None :
                sub=re.sub(r' ', " notadd", matchObj1.group(2))
                list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
                list_text[i] = re.sub(r'not ','', list_text[i],1)
                #list_text[i] = re.sub(',',' ', matchObj1.group(3))
                sent = sent + list_text[i] + ' '
        
            elif matchObj2 != None :
                sub=re.sub(r' ', " notadd", matchObj2.group(2))
                list_text[i] = re.sub(matchObj2.group(2) ,sub, list_text[i])
                list_text[i] = re.sub(r'never ','', list_text[i],1)
                #list_text[i] = re.sub(',',' ', matchObj1.group(3))
                sent = sent + list_text[i] + ' '
        
            else:
                sent = sent + list_text[i] + ' '
        
        text_clean=preprocess_reviews(sent)
        #print(text_clean)
    
        text_cloud1=text_cloud1+text_clean
        reviewsTEXT_clean[ind]['text']=text_clean
        reviewsTEXT_clean[ind]['text_split']=text_clean.split()
        ps = PorterStemmer()
        stem=[]
        for w in reviewsTEXT_clean[ind]['text_split']:
            stem.append(ps.stem(w))
        reviewsTEXT_clean[ind]['text_stem']=stem
        reviewsTEXT_clean[ind]['freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['text_stem'])
        
        
    stop_words = set(stopwords.words('english')) 
    ps = PorterStemmer()
    stop_words_stem=[]
    for w in stop_words:
        stop_words_stem.append(ps.stem(w))
    negWords=set(["wouldn't",'isn','wasn',"weren't", "haven't", "hasn't", "couldn't", "isn't", 'doesn','hasn',"mustn't", 'mightn', 'shan', 'no', "wasn't",'aren', "didn't", "hadn't","don't",'nor',"won't",'weren',"doesn't","needn't", 'shouldn',"mightn't","shan't", 'wouldn',"shouldn't",'hadn'])
    negWords_stem=[]
    for w in negWords:
        negWords_stem.append(ps.stem(w))
    d=set(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','à','{sigh}'])
    stopWords_stem=set(stop_words_stem)-set(negWords_stem)
    stopWords_stem.update(d)
    stopWords_stem.add('was')
    
    from stemming.porter2 import stem
    #remove stopwords 
    CleanReviews=[]
    allReviews=[x['text'] for x in reviewsTEXT_clean]
    for sentence in allReviews:
        sentenceSTEM=" ".join([stem(word) for word in sentence.split(" ")])
        reviewWOsw=' '.join(filter(lambda x: x not in stopWords_stem,  sentenceSTEM.split()))
        CleanReviews.append(reviewWOsw)
        
    #rvs2= [" ".join([stem(word) for word in sentence.split(" ")]) for sentence in CleanReviews]
    rvs2=CleanReviews
    twoWords = {}
    for i in rvs2:
        twoWdLs=generate_ngrams(i, n)
        for j in twoWdLs:
            if len(j.split())==n:
                if j in twoWords:
                    twoWords[j]=twoWords[j]+1
                else:
                    twoWords[j]=1
    #twoWords

    bigrams=sorted(twoWords.items(), key=lambda x: x[1],reverse=True)
    return [x for x in bigrams if xxx in x[0]]


In [86]:
beer1=bigramFreq(dat_night_1star,'beer',2)

In [87]:
beer2=bigramFreq(dat_night_2star,'beer',2)

In [88]:
beer3=bigramFreq(dat_night_3star,'beer',2)

In [89]:
beer4=bigramFreq(dat_night_4star,'beer',2)

In [90]:
beer5=bigramFreq(dat_night_5star,'beer',2)

In [81]:
#beer1

In [82]:
#beer2

In [83]:
#beer3

In [84]:
#beer4

In [85]:
#beer5

In [93]:
from pandas import DataFrame
df1=DataFrame(beer1)
df2=DataFrame(beer2)
df3=DataFrame(beer3)
df4=DataFrame(beer4)
df5=DataFrame(beer5)
df12=pd.concat([df1,df2],axis=1)
df123=pd.concat([df12,df3],axis=1)
df1234=pd.concat([df123,df4],axis=1)
beerdf=pd.concat([df1234,df5],axis=1)

In [94]:
beerdf.columns = ['beer1', 'beer1freq','beer2', 'beer2freq','beer3', 'beer3freq','beer4', 'beer4freq','beer5', 'beer5freq']

In [95]:
DataFrame.to_csv(beerdf,"beerBigram.csv",index=False)

In [96]:
casino1=bigramFreq(dat_night_1star,'casino',2)
casino2=bigramFreq(dat_night_2star,'casino',2)
casino3=bigramFreq(dat_night_3star,'casino',2)
casino4=bigramFreq(dat_night_4star,'casino',2)
casino5=bigramFreq(dat_night_5star,'casino',2)

In [97]:
from pandas import DataFrame
df1=DataFrame(casino1)
df2=DataFrame(casino2)
df3=DataFrame(casino3)
df4=DataFrame(casino4)
df5=DataFrame(casino5)
df12=pd.concat([df1,df2],axis=1)
df123=pd.concat([df12,df3],axis=1)
df1234=pd.concat([df123,df4],axis=1)
casinodf=pd.concat([df1234,df5],axis=1)

In [98]:
casinodf.columns = ['casino1', 'casino1freq','casino2', 'casino2freq','casino3', 'casino3freq','casino4', 'casino4freq','casino5', 'casino5freq']
DataFrame.to_csv(casinodf,"casinoBigram.csv",index=False)

In [100]:
casino1=bigramFreq(dat_night_1star,'casino',3)
casino2=bigramFreq(dat_night_2star,'casino',3)
casino3=bigramFreq(dat_night_3star,'casino',3)
casino4=bigramFreq(dat_night_4star,'casino',3)
casino5=bigramFreq(dat_night_5star,'casino',3)

In [101]:
from pandas import DataFrame
df1=DataFrame(casino1)
df2=DataFrame(casino2)
df3=DataFrame(casino3)
df4=DataFrame(casino4)
df5=DataFrame(casino5)
df12=pd.concat([df1,df2],axis=1)
df123=pd.concat([df12,df3],axis=1)
df1234=pd.concat([df123,df4],axis=1)
casinodf=pd.concat([df1234,df5],axis=1)

In [102]:
casinodf.columns = ['casino1', 'casino1freq','casino2', 'casino2freq','casino3', 'casino3freq','casino4', 'casino4freq','casino5', 'casino5freq']
DataFrame.to_csv(casinodf,"casinoTrigram.csv",index=False)

In [104]:
#cocktail
cocktail1=bigramFreq(dat_night_1star,'cocktail',2)
cocktail2=bigramFreq(dat_night_2star,'cocktail',2)
cocktail3=bigramFreq(dat_night_3star,'cocktail',2)
cocktail4=bigramFreq(dat_night_4star,'cocktail',2)
cocktail5=bigramFreq(dat_night_5star,'cocktail',2)
from pandas import DataFrame
df1=DataFrame(cocktail1)
df2=DataFrame(cocktail2)
df3=DataFrame(cocktail3)
df4=DataFrame(cocktail4)
df5=DataFrame(cocktail5)
df12=pd.concat([df1,df2],axis=1)
df123=pd.concat([df12,df3],axis=1)
df1234=pd.concat([df123,df4],axis=1)
cocktaildf=pd.concat([df1234,df5],axis=1)

In [106]:
cocktaildf.columns = ['cocktail1', 'cocktail1freq','cocktail2', 'cocktail2freq','cocktail3', 'cocktail3freq','cocktail4', 'cocktail4freq','cocktail5', 'cocktail5freq']
DataFrame.to_csv(cocktaildf,"cocktailBigram.csv",index=False)


In [107]:
cocktaildf

,cocktail1,cocktail1freq,cocktail2,cocktail2freq,cocktail3,cocktail3freq,cocktail4,cocktail4freq,cocktail5,cocktail5freq
0,cocktail waitress,114.0,shrimp cocktail,76.0,shrimp cocktail,106.0,shrimp cocktail,194.0,craft cocktail,358
1,shrimp cocktail,48.0,cocktail waitress,76.0,cocktail waitress,91.0,cocktail menu,183.0,shrimp cocktail,337
2,order cocktail,40.0,order cocktail,43.0,cocktail menu,66.0,cocktail waitress,172.0,great cocktail,296
3,cocktail server,30.0,cocktail order,29.0,cocktail good,58.0,specialti cocktail,156.0,cocktail menu,271
4,one cocktail,25.0,cocktail menu,28.0,specialti cocktail,54.0,craft cocktail,148.0,cocktail waitress,215
5,cocktail menu,21.0,specialti cocktail,28.0,order cocktail,50.0,great cocktail,105.0,specialti cocktail,195
6,cocktail order,20.0,cocktail sauc,26.0,good cocktail,42.0,cocktail good,99.0,cocktail great,154
7,notadda notaddcocktail,17.0,cocktail good,23.0,cocktail sauc,40.0,good cocktail,88.0,amaz cocktail,146
8,specialti cocktail,16.0,notadda notaddcocktail,21.0,craft cocktail,33.0,signatur cocktail,87.0,cocktail bar,135
9,cocktail sauc,15.0,one cocktail,20.0,beer cocktail,29.0,enjoy cocktail,80.0,best cocktail,133


In [108]:
#cocktail 3
cocktail1=bigramFreq(dat_night_1star,'cocktail',3)
cocktail2=bigramFreq(dat_night_2star,'cocktail',3)
cocktail3=bigramFreq(dat_night_3star,'cocktail',3)
cocktail4=bigramFreq(dat_night_4star,'cocktail',3)
cocktail5=bigramFreq(dat_night_5star,'cocktail',3)
from pandas import DataFrame
df1=DataFrame(cocktail1)
df2=DataFrame(cocktail2)
df3=DataFrame(cocktail3)
df4=DataFrame(cocktail4)
df5=DataFrame(cocktail5)
df12=pd.concat([df1,df2],axis=1)
df123=pd.concat([df12,df3],axis=1)
df1234=pd.concat([df123,df4],axis=1)
cocktaildf=pd.concat([df1234,df5],axis=1)
cocktaildf.columns = ['cocktail1', 'cocktail1freq','cocktail2', 'cocktail2freq','cocktail3', 'cocktail3freq','cocktail4', 'cocktail4freq','cocktail5', 'cocktail5freq']
DataFrame.to_csv(cocktaildf,"cocktailTrigram.csv",index=False)

In [109]:
cocktaildf

,cocktail1,cocktail1freq,cocktail2,cocktail2freq,cocktail3,cocktail3freq,cocktail4,cocktail4freq,cocktail5,cocktail5freq
0,order shrimp cocktail,10.0,order shrimp cocktail,9.0,order shrimp cocktail,11.0,order shrimp cocktail,19.0,hand craft cocktail,51
1,one cocktail waitress,7.0,one cocktail waitress,7.0,shrimp cocktail appet,7.0,hand craft cocktail,18.0,best cocktail ever,36
2,servic cocktail waitress,5.0,notadda notaddcocktail notaddwaitress,6.0,shrimp cocktail good,6.0,happi hour cocktail,18.0,one best cocktail,29
3,tabl cocktail waitress,4.0,bartend cocktail waitress,5.0,beer wine cocktail,6.0,cocktail happi hour,17.0,food great cocktail,27
4,cocktail tast like,4.0,cocktail sauc shrimp,4.0,cocktail realli good,6.0,cocktail this place,16.0,cocktail this place,27
5,night cocktail waitress,4.0,cocktail tast like,4.0,great cocktail menu,5.0,specialti cocktail menu,16.0,jumbo shrimp cocktail,22
6,shrimp cocktail good,4.0,shrimp cocktail appet,4.0,cocktail peopl watch,5.0,cocktail realli good,15.0,happi hour cocktail,21
7,this place cocktail,4.0,cocktail pretti good,4.0,cocktail pretti good,5.0,cocktail sauc horseradish,13.0,craft cocktail bar,19
8,downtown cocktail room,4.0,jumbo shrimp cocktail,4.0,happi hour cocktail,5.0,jumbo shrimp cocktail,12.0,cocktail high recommend,19
9,shrimp cocktail shrimp,3.0,cocktail friend order,4.0,friend cocktail waitress,5.0,one signatur cocktail,11.0,great cocktail great,19


In [ ]:
#wait 2
wait1=bigramFreq(dat_night_1star,'wait',2)
wait2=bigramFreq(dat_night_2star,'wait',2)
wait3=bigramFreq(dat_night_3star,'wait',2)
wait4=bigramFreq(dat_night_4star,'wait',2)
wait5=bigramFreq(dat_night_5star,'wait',2)
from pandas import DataFrame
df1=DataFrame(wait1)
df2=DataFrame(wait2)
df3=DataFrame(wait3)
df4=DataFrame(wait4)
df5=DataFrame(wait5)
df12=pd.concat([df1,df2],axis=1)
df123=pd.concat([df12,df3],axis=1)
df1234=pd.concat([df123,df4],axis=1)
waitdf=pd.concat([df1234,df5],axis=1)
waitdf.columns = ['wait1', 'wait1freq','wait2', 'wait2freq','wait3', 'wait3freq','wait4', 'wait4freq','wait5', 'wait5freq']
DataFrame.to_csv(waitdf,"waitBigram.csv",index=False)

In [ ]:
#wait 3
wait1=bigramFreq(dat_night_1star,'wait',2)
wait2=bigramFreq(dat_night_2star,'wait',2)
wait3=bigramFreq(dat_night_3star,'wait',2)
wait4=bigramFreq(dat_night_4star,'wait',2)
wait5=bigramFreq(dat_night_5star,'wait',2)
from pandas import DataFrame
df1=DataFrame(wait1)
df2=DataFrame(wait2)
df3=DataFrame(wait3)
df4=DataFrame(wait4)
df5=DataFrame(wait5)
df12=pd.concat([df1,df2],axis=1)
df123=pd.concat([df12,df3],axis=1)
df1234=pd.concat([df123,df4],axis=1)
waitdf=pd.concat([df1234,df5],axis=1)
waitdf.columns = ['wait1', 'wait1freq','wait2', 'wait2freq','wait3', 'wait3freq','wait4', 'wait4freq','wait5', 'wait5freq']
DataFrame.to_csv(waitdf,"waitBigram.csv",index=False)